In [ ]:
! pip install graph-notebook --quiet

In [ ]:
! pip install pandas python-dotenv SQLAlchemy psycopg2-binary --quiet

**LOAD ENV**

In [1]:
import os
%load_ext dotenv
%dotenv

DATASET_PATH=os.getenv("DATASET_PATH")
JSON_DATASET_PATH=os.getenv("JSON_DATASET_PATH")
POSTGRESDB_URL=os.getenv("POSTGRESDB_URL")
HASURA_RESTAPI_HOST = os.getenv("HASURA_RESTAPI_HOST")
# NEO4J_HOST=os.getenv("NEO4J_HOST")
# NEO4J_USER=os.getenv("NEO4J_USER")
# NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")

**LOAD graph-notebook**

In [2]:
%load_ext graph_notebook.magics

Could not find a valid configuration. Do not forget to validate your settings using %graph_notebook_config.


In [3]:
%%graph_notebook_config
{
  "host": "localhost",
  "port": 7687,
  "ssl": false,
  "neo4j": {
    "username": "neo4j",
    "password": "neo4jpassword",
    "auth": true,
    "database": ""
  }
}

set notebook config to:
{
  "host": "localhost",
  "port": 7687,
  "proxy_host": "",
  "proxy_port": 8182,
  "ssl": false,
  "ssl_verify": true,
  "sparql": {
    "path": ""
  },
  "gremlin": {
    "traversal_source": "g",
    "username": "",
    "password": "",
    "message_serializer": "graphsonv3"
  },
  "neo4j": {
    "username": "neo4j",
    "password": "neo4jpassword",
    "auth": true,
    "database": null
  }
}


In [ ]:
%%oc bolt

Match (n)

## Setup Postgres

In [ ]:
print(POSTGRESDB_URL)

In [ ]:
from sqlalchemy import create_engine, MetaData
engine = create_engine(POSTGRESDB_URL)
metadata = MetaData()
# reflect the database schema to the metadata
metadata.reflect(bind=engine)

In [ ]:
from sqlalchemy import text as sqltext

def run_sql_script(path):

    sql = open(path).read();
    with engine.connect() as conn:
        escaped_sql = sqltext(sql)
        result = conn.execute(escaped_sql)
        conn.commit()
        return result
    
def run_sql_query(sql):
    with engine.connect() as conn:
        escaped_sql = sqltext(sql)
        result = conn.execute(escaped_sql)
        conn.commit()
        return result

In [ ]:
from sqlalchemy import text as sqltext
table = run_sql_script("./sql/northwind_pg.sql")
table.keys

## Exporting Nodes to CSV using GraphQL

    // GRAPHQL QUERY
    query Nodes {
      customers {
        customerid
        customername
      }
      categories {
        categoryid
        categoryname
      }
      orders {
        orderid
        orderdate
      }
      products {
        productid
        productname
      }
    }



In [ ]:
import shutil
# delete exsisting path for neptune dataset
shutil.rmtree(DATASET_PATH)

# mkdir
isExist = os.path.exists(DATASET_PATH)
if not isExist:
   os.makedirs(DATASET_PATH)

In [12]:
NODES_API = "nodes"
EDGES_API = "edges"

In [ ]:
import pandas as pd
import requests

url =  "{host}/{api}".format(host=HASURA_RESTAPI_HOST,api=NODES_API)
response = requests.get(url)
response_json = response.json()
response_json["customers"][0]

In [ ]:
datasets = ['customers','categories','orders','products']

# Nodes
# :ID,  name:String,  age:Int,  lang:String,  :LABEL
                    
for d in datasets:
    json_arr = response_json[d]
    df = pd.DataFrame(json_arr)
    if(d == 'orders'):
        df['orderdate'] = pd.to_datetime(df['orderdate']).dt.date
        
    df.to_csv('{path}/{dataset}.csv'.format(path=DATASET_PATH,dataset=d),index=False)

In [ ]:
df = pd.read_csv('{path}/{dataset}.csv'.format(path=DATASET_PATH,dataset='orders'))
df

Upload Data to Neo4j: https://neo4j.com/developer/guide-importing-data-and-etl/

## Import Data to Neo4j (Nodes)

### Orders

In [ ]:
%%oc bolt

LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/final/csv-data/orders.csv' AS row
MERGE (order:Order {orderID: row.orderid})
  ON CREATE SET order.orderDate = row.orderdate;

In [4]:
%%oc bolt
MATCH (o:Order) return o LIMIT 5;

### Products

In [ ]:
%%oc bolt

LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/final/csv-data/products.csv' AS row
MERGE (product:Product {productID: row.productid})
  ON CREATE SET product.productName = row.productname;

In [5]:
%%oc bolt
MATCH (p:Product) return p LIMIT 5;

### Categories

In [ ]:
%%oc bolt

LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/final/csv-data/categories.csv' AS row
MERGE (c:Category {categoryID: row.categoryid})
  ON CREATE SET c.categoryName = row.categoryname;

In [6]:
%%oc bolt
MATCH (c:Category) return c LIMIT 5;

### Customers

In [9]:
%%oc bolt

LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/csv-data/customers.csv' AS row
MERGE (c:Customer {customerID: row.customerid})
  ON CREATE SET c.customerName = row.customername;

In [10]:
%%oc bolt
MATCH (c:Customer) return c LIMIT 5;

## Exporting Edges to CSV using GraphQL
    // GRAPHQL Query 
     query Edges{
      customers {
        customerid
        orders {
          orderid
        }
      }
      orders {
        orderid
        orderdetails {
          orderdetailid
          product {
            productid
          }
        }
      }
      products {
        productid
        category {
          categoryid
        }
      }
    }

### Customer and Products
![Popular Products](https://ik.imagekit.io/ggp/4c559e14-baaa-4d77-ad62-f3f52deab74a_q6scT_bsop)

In [13]:
import pandas as pd
import requests

url =  "{host}/{api}".format(host=HASURA_RESTAPI_HOST,api=EDGES_API)
response = requests.get(url)
response_json = response.json()
response_json["products"][0]

{'productid': 1, 'category': {'categoryid': 1}}

In [14]:
# :Purchased

data = []
data.append(("from","to"))
json_arr = response_json['customers']
for elem in json_arr:
    customerid = elem['customerid']
    for o in elem['orders']:
        data.append((customerid,o['orderid']))

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df = df[1:]
df.to_csv('{path}/{dataset}.csv'.format(path=DATASET_PATH,dataset="purchased"),index=False,header=True)
   

In [15]:
df = pd.read_csv('{path}/{dataset}.csv'.format(path=DATASET_PATH,dataset='purchased'))
df

,from,to
0,2,10308
1,3,10365
2,4,10355
3,4,10383
4,5,10278
...,...,...
191,88,10420
192,89,10269
193,89,10344
194,90,10248


In [16]:
%%oc bolt

// Create relationships between customers and orders
LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/csv-data/purchased.csv' AS row
MATCH (customer:Customer {customerID: row.from})
MATCH (order:Order {orderID: row.to})
MERGE (customer)-[pu:PURCHASED]->(order)

In [17]:
%%oc bolt

MATCH (p)-[r:PURCHASED]->(o) RETURN p,r,o LIMIT 25;

In [18]:
# :Product
# {'orderid': 10248,
#  'orderdetails': [{'orderdetailid': 1, 'product': {'productid': 11}},
#   {'orderdetailid': 2, 'product': {'productid': 42}},
#   {'orderdetailid': 3, 'product': {'productid': 72}}]}


data = []
data.append(("from","to"))
json_arr = response_json['orders']
for elem in json_arr:
    orderid = elem['orderid']
    for od in elem['orderdetails']:
        productid= od['product']['productid']
        data.append((orderid,productid))

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df = df[1:]
df.to_csv('{path}/{dataset}.csv'.format(path=DATASET_PATH,dataset="order_product"),index=False,header=True)

In [19]:
df = pd.read_csv('{path}/{dataset}.csv'.format(path=DATASET_PATH,dataset='order_product'))
df

,from,to
0,10248,11
1,10248,42
2,10248,72
3,10249,14
4,10249,51
...,...,...
513,10442,11
514,10442,54
515,10442,66
516,10443,11


In [20]:
%%oc bolt

// Create relationships between corder and product
LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/csv-data/order_product.csv' AS row
MATCH (order:Order {orderID: row.from})
MATCH (product:Product {productID: row.to})
MERGE (order)-[pr:PRODUCT]->(product)

In [22]:
%%oc bolt

MATCH (o)-[pr:PRODUCT]->(p) RETURN o,pr,p LIMIT 25;

In [23]:
# :PartOf
#  {'productid': 1, 'category': {'categoryid': 1}}

data = []
data.append(("from","to"))
json_arr = response_json['products']
for elem in json_arr:
    productid = elem['productid']
    categoryid = elem['category']['categoryid']
    data.append((productid,categoryid))

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df = df[1:]
df.to_csv('{path}/{dataset}.csv'.format(path=DATASET_PATH,dataset="partof"),index=False,header=True)

In [24]:
df = pd.read_csv('{path}/{dataset}.csv'.format(path=DATASET_PATH,dataset='partof'))
df

In [4]:
%%oc bolt

// Create relationships between product and category
LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/csv-data/partof.csv' AS row
MATCH (product:Product {productID: row.from})
MATCH (category:Category {categoryID: row.to})
MERGE (product)-[pr:PARTOF]->(category)

In [5]:
%%oc bolt

MATCH (p)-[pof:PARTOF]->(c) RETURN p,pof,c LIMIT 25;

## Create Ratings

![Create Ratings](https://ik.imagekit.io/ggp/33d26c02-b28c-4d82-9be4-ffb024550153_1NcVSJLAC)

In [6]:
%%oc bolt

MATCH (c:Customer)-[:PURCHASED]->(o:Order)-[:PRODUCT]->(p:Product)
WITH c, count(p) as total
MATCH (c)-[:PURCHASED]->(o:Order)-[:PRODUCT]->(p:Product)
WITH c, total,p, count(o)*1.0 as orders
MERGE (c)-[rated:RATED]->(p)
ON CREATE SET rated.rating = orders/total
ON MATCH SET rated.rating = orders/total
WITH c.companyName as company, p.productName as product, orders, total, rated.rating as rating
ORDER BY rating DESC
RETURN company, product, orders, total, rating LIMIT 10

In [7]:
%%oc bolt 

MATCH (me:Customer)-[r:RATED]->(p:Product)
WHERE me.customerID="48"
RETURN p.productName, r.rating limit 10